In [1]:
%load_ext autoreload
%autoreload 2

# Guarded Output Parsers

With `OutputParser`s (see `./output_parsers.ipynb`), we can prompt a LM to format its completion with a desired schema. The `PydanticOutputParser` is the most powerful parser, allowing us to specify arbitrary json schema.

Unfortunately, small models to date don't have capacity to generate well-formed, schema-adherent json while large models still sometimes fail.

In this notebook, we showcase a "Guarded" `OutputParser` which can be dropped in for an `OutputParser` in an `LLMChain`. It will catch errors at parsing time and try resolve them, initially by re-invoking an LLM

Below are some examples:

In [1]:
from pydantic import BaseModel, Field
from typing import List

from langchain.chains import LLMChain
from langchain.guardrails.parsing import RetriableOutputParser
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser, OutputParserException
from langchain.prompts import PromptTemplate

## 1st example

In [2]:
# Pydantic data structure.
class FloatArray(BaseModel):
    values: List[float] = Field(description="list of floats")

# Query that will populate the data structure.
float_array_query = "Write out a few terms of fiboacci."

In [3]:
# Declare a parser and prompt.
parser = PydanticOutputParser(pydantic_object=FloatArray)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Currently, the parser is set on the prompt template.
prompt.output_parser = parser

In [4]:
# For demonstration's sake, we'll use a "small" model that probably won't generate json properly.
llm_chain = LLMChain(
    prompt=prompt,
    llm=OpenAI(model_name="text-curie-001"),
    verbose=True)

try:
    llm_chain.predict(query=float_array_query)
except OutputParserException as e:
    print("Dang!")
    print(e)



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below. For example, the object {"foo": ["bar", "baz"]} conforms to the schema {"foo": {"description": "a list of strings field", "type": "string"}}.

Here is the output schema:
```
{"values": {"description": "list of floats", "type": "array"}}
```
Write out a few terms of fiboacci.

Dang!
Failed to parse FloatArray from completion 
Fiboacci is a sequence of numbers that are the sum of the previous two numbers in the sequence.. Got: Expecting value: line 1 column 1 (char 0)


In [5]:
# We can replace the parser with a guarded parser that tries to fix errors with a bigger model.
guarded_parser = RetriableOutputParser(
    parser=parser, retry_llm=OpenAI(model_name="text-davinci-003"))
prompt.output_parser = guarded_parser

llm_chain.predict(query=float_array_query)



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below. For example, the object {"foo": ["bar", "baz"]} conforms to the schema {"foo": {"description": "a list of strings field", "type": "string"}}.

Here is the output schema:
```
{"values": {"description": "list of floats", "type": "array"}}
```
Write out a few terms of fiboacci.


> Finished chain.


FloatArray(values=[1.0, 2.0, 3.0, 5.0, 8.0, 13.0, 21.0, 34.0, 55.0, 89.0])

This example is demonstrative though. If your goal is to generate data structures, probably you'll want to start a large enough model.